[스타트코딩의 이것이 진짜 크롤링이다 유튜브 영상을 보고 실습, 응용 해본 자료입니다.](https://www.youtube.com/watch?v=qRU94vtUb7c&list=PLNO7MWpu0eeUFdGMirV8_EkiLETqj8xA4&index=9)

```작성일: 2022.01.08``` 
```작성자: 박유정```


목표: 네이버 쇼핑 홈페이지에서 상품정보와 리뷰 데이터를 가져오기   

크롤링 로직
1. 네이버 쇼핑 홈페이지에 접속한다.
2. 검색어 입력창을 선택한다.
3. 검색어를 입력한다.
4. 엔터를 누른다.
5. 스크롤을 끝까지 내린다. (일정 범위까지만 내릴 수도 있음)
6. 제품정보 레이아웃을 선택한다.
7. 원하는 제품 정보 요소들을 선택한다. 제품명, 가격, 상세 페이지 URL 등)
8. 제품 정보를 저장할 csv 파일을 연다.
9. for문을 통해 제품별 정보를 리스트 형태로 가져온다.
10. csv 파일로 저장한다.

# 라이브러리 가져오기

In [17]:
# 크롤링 모듈 임포트
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import csv
import urllib.request
import openpyxl

# 웹 브라우저 열기

- browser.get(): url 주소 받아서 열기

In [6]:
#  브라우저 생성
browser = webdriver.Chrome('C:/Users/data_test/chromedriver.exe')

# 접속할 페이지 주소 입력
url = "https://shopping.naver.com/home/p/index.naver"

# 웹사이트 열기
browser.get(url)

C:\Users\skygg\AppData\Local\Temp/ipykernel_4368/4268051727.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:/Users/data_test/chromedriver.exe')


# 제품을 검색하고 제품정보 수집하기

## 검색창 선택
- find_element_by_css_selector(): 요소 찾기, 여기에서는 검색어 입력창
- click(): 해당 영역 클릭

In [7]:
# 검색창 클릭
search = browser.find_element_by_css_selector("input.co_srh_input._input")
time.sleep(2) # 브라우저가 뜨는 시간을 기다려줘야 함
search.click()

C:\Users\skygg\AppData\Local\Temp/ipykernel_4368/4067961810.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = browser.find_element_by_css_selector("input.co_srh_input._input")


## 검색창에 인자 전달하기 

- send_keys(): 검색어나 키보드의 키를 PC에 전달

- 전달할 수 있는 키보드 키값
> Kyes.ENTER, Keys.ARROW_DOWN, Keys.ARROW_LEFT, Keys.ARROW_RIGHT, Keys.ARROW_UP,
Keys.BACK_SPACE, Keys.CONTROL, Keys.ALT, Keys.DELETE, Keys.TAB, Keys.SPACE,
Keys.SHIFT, Keys.EQUALS, Keys.ESCAPE, Keys.HOME, Keys.INSERT, Keys.PAGE_UP,
Keys.PAGE_DOWN,Keys.F1, Keys.F2, Keys.F3, Keys.F4, Keys.F5, Keys.F6, Keys.F7,
Keys.F8, Keys.F9 Keys.F10, Keys.F11, Keys.F12

In [8]:
# 검색어 입력
keyword = input('검색어를 입력하세요:')
search.send_keys(keyword)

# ENTER 누르기
search.send_keys(Keys.ENTER)

검색어를 입력하세요:무선 청소기


## 스크롤 내리기

fn + F12(개발자 도구) -> Console -> window.scrollY 명령어를 실행시켜보세요. 현재 스크롤된 높이를 반환합니다.
- window.scrollY: 스크롤된 높이를 알 수 있는 자바스크립트 명령어
- execute_script(): 자바스크립트 명령어 실행
- body: HTML 태그, 브라우저에서 모든 콘텐츠를 포함하는 영역

In [9]:
# 스크롤 전 높이
before_h = browser.execute_script("return window.scrollY")

# 무한 스크롤
while True:
    # 맨 아래로 스크롤을 내린다.
    browser.find_element_by_css_selector("body").send_keys(Keys.END)
    
    # 스크롤 사이 페이지 로딩을 기다림
    time.sleep(2)
    
    # 스크롤 후 높이
    after_h = browser.execute_script("return window.scrollY")
    
    if after_h == before_h: # 끝까지 다 내렸다면
        break # 스크롤 그만
    before_h = after_h

C:\Users\skygg\AppData\Local\Temp/ipykernel_4368/2655887189.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_css_selector("body").send_keys(Keys.END)


## 제품정보를 리스트 형태로 가져오고 파일에 쓰기

※ 유의할 점
- driver.find_element_by_css_selector(): 찾는 css가 1개 일 때
- driver.find_elements_by_css_selector(): 찾는 css가 여러개 일 때
- text: 텍스트 정보 가져오기
- get_attribute(): 속성값 가져오기

In [20]:
# 파일 열기(파일명, 모드(쓰기모드), 인코딩 종류, 줄바꿈 문자 없애기)
f = open("prouct data.csv", 'w', encoding='CP949', newline='')

# 상품 정보 div 가져오기
items = browser.find_elements_by_css_selector(".basicList_info_area__17Xyo")

# 제품별 정보 가져오기
for item in items:
    # 제품명
    name = item.find_element_by_css_selector(".basicList_title__3P9Q7").text 
    # 가격
    try:
        price = item.find_element_by_css_selector(".price_num__2WUXn").text 
    except: # 가격 정보가 없을 경우 제외처리 (이 부분이 없으면 오류가 남)
        price = '판매중단'
    
    # 카테고리
    category1 = browser.find_element_by_css_selector("div.basicList_depth__2QIie > a:nth-child(2)").text # 대분류
    category2 = browser.find_element_by_css_selector("div.basicList_depth__2QIie > a:nth-child(3)").text # 중분류
    category3 = browser.find_element_by_css_selector("div.basicList_depth__2QIie > a:nth-child(4)").text # 소분류
    
    # 제품 특징
    product_features1 = browser.find_element_by_css_selector(".basicList_detail_box__3ta3h ").text
    
    # 상세 페이지 URL
    link = item.find_element_by_css_selector(".basicList_title__3P9Q7 > a").get_attribute('href') 
    print(name, price, category1,category2, category3, product_features, link )
    
    # 데이터 쓰기
    csvwriter = csv.writer(f)
    csvwriter.writerow([name, price, category1,category2, category3, product_features,link])

# 파일 닫기
# f.close()

C:\Users\skygg\AppData\Local\Temp/ipykernel_4368/3457409356.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  items = browser.find_elements_by_css_selector(".basicList_info_area__17Xyo")
C:\Users\skygg\AppData\Local\Temp/ipykernel_4368/3457409356.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  category1 = browser.find_element_by_css_selector("div.basicList_depth__2QIie > a:nth-child(2)").text # 대분류
C:\Users\skygg\AppData\Local\Temp/ipykernel_4368/3457409356.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  category2 = browser.find_element_by_css_selector("div.basicList_depth__2QIie > a:nth-child(3)").text # 중분류
C:\Users\skygg\AppData\Local\Temp/ipykernel_4368/3457409356.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  category3 = browser.find_element_by_css_sele

로이드미 X30 Pro 469,410원 생활가전 청소기 무선청소기 품목 : 핸디스틱청소기|형태 : 상중심|집진방식 : 싸이클론|사용시간 : 1시간10분|청소방식 : 흡입전용|충전시간 : 2시간30분|배터리용량 : 2,500mAh|배터리종류 : 리튬이온|, , , , , , , https://adcr.naver.com/adcr?x=PzCdSeg8KgL75g2k89w0Sf///w==kieSDHHaMC6sIVvgzQSiFPtOOwtWZ6oKlSefCzZlzn9LRJIm2tTjZQdoi+Q3AIt6DRlYf/KEQPWqFqJ+tQyASqoEuwN9MxjS0wKCwgezHReLO8PqSZQoRENi1DRm9/4rMgNo0sLaGr1mOW6/5PjV7UvZmf/pCy33HneneB1KFGMVje6eoTyrpDglVibq/dChtfNF5W5rBFp9p/3HyippBjmk/cpi0Yz/4RsLAi16o6194JUc6qhZZmgtTw9iyxcNjwz4Mzxf0KKleYBrQHCfEpj3ie1WU3YXhWYBR6XDGpUS3YMJGKC2vodiF0NAPmol0IFgD2TCgFhDvi7woA+e2/RMSiLKzxlbZHB4uk+VB1bv5s8WCpxNYXI5cMI7SllUZy5PdPFsy1gofbkxrqFjvSM7jHKgcJrICIu2PzULLEcuJKT3Qu+BUG/HAERV16+j1nccCWoSds2QZWxQ1OyqjZ6eUVmrXzF0N1KnYfdywe3DgNL4fVPF2uc84pYUHn0nOOmvcJ+K7ZAKTIXdrRKyaF8oVgyRVMTKBLYrDdsTZ9ePpkk2ka9f6XC0A/M3PfZ0bbfi5chSE0HKIlM8WBfxjdanoa62jSEdmyMkz7yjJ5/9z2iAwMB/sDDKwSRIe1oSCAM+dLeZ3LhveUgFCs1inaLiaCF+2+wh30Vsr77kjN+1R6PlsOQZ2OKagazQQhA/OLgbZ3fx8crKEBupBvriQy0m0l4gKi/1H1RDZoIKsnP89bs47wdA6Rp2k1KsUKg+jJToe0AlDYjV2raMLSz

비브르 스톰파워 VE30 173,000원 생활가전 청소기 무선청소기 품목 : 핸디스틱청소기|형태 : 상중심|집진방식 : 싸이클론|사용시간 : 1시간10분|청소방식 : 흡입전용|충전시간 : 2시간30분|배터리용량 : 2,500mAh|배터리종류 : 리튬이온|, , , , , , , https://cr.shopping.naver.com/adcr.nhn?x=NTj0BFHfmfT%2FXPBPlsOi6P%2F%2F%2Fw%3D%3DsBw2k6IHzf1g3I201yheIgk%2BlNvK0UGEaBccfHpK5EWaJ5ltDmER3oclazeOpToBHwZf%2BuTyHVhnU6w7SKqZ6vRc%2B%2BCLa6%2FcsJq%2FYL0h%2F%2B0ZQm4SLs3i%2FjGGPR43wmWtwFGhnHFn6I2VJH6fQb%2FD%2BstyGaSr%2B0dRu37W7MMIB4NFk9Io6iZoAfEBQfsbkhBMBj38CBhRT8g2XS4LOvCOq%2B38y8PM6NF45tD5kqxhOgdtjhT3q51gOqoSu%2FQESHdAPjis8avLprLsNKR2b4t3hAVWUgCnMWAAqDjgi4wBmkEsYYDUUxQlBVqXtVrGEeR0ehmeWtJmvw8M90EK%2Bzy5a1yVdN3LpKf%2FbKuiZi7z1XJTc6USRY4FcQqvUwhh5kKz9n8%2FWvHuyRDgcBs8jfiX6%2F1d%2BqILAG09hjAR1K3qLYkBpVTswc9DlIgwiD6BiP8U33pbiwTfiPvSYZMxFwycMdARzTktOJsG39rC0oovAKqeZ5iVWLM9IxiHXJD%2FPDyuto64ifB9DVeZ8ffla9GlFdjIhkfW%2FAKfWa3zaZqZlKBskw31ly4bzVRyRRXxQfA1Njr%2Fvhuz%2FM2bOeEg7k3nONcQ%2FMHyoDYo04YICo9gtZ7U222mZrqe67eJQmDyUuMnZhWw3%2BIByOa0jCVNQVnCDoA%3D%3D&nvMid=21629247611&catId=50002350
LG전자 오브제컬렉

삼성전자 비스포크제트 VS20A956A3 + 물걸레 783,020원 생활가전 청소기 무선청소기 품목 : 핸디스틱청소기|형태 : 상중심|집진방식 : 싸이클론|사용시간 : 1시간10분|청소방식 : 흡입전용|충전시간 : 2시간30분|배터리용량 : 2,500mAh|배터리종류 : 리튬이온|, , , , , , , https://cr.shopping.naver.com/adcr.nhn?x=SuXAeCGNIG0rJtP%2B6zxmxf%2F%2F%2Fw%3D%3DsWMWnUUgbL19hmctNwRMcnVmczmfQ%2BtmXmC5dQPFCs9uJ5ltDmER3oclazeOpToBH05nbgoNRccf495ZjhnVWjI3mCAkiDePdzgQMBti6LBNQm4SLs3i%2FjGGPR43wmWtwFGhnHFn6I2VJH6fQb%2FD%2BstyGaSr%2B0dRu37W7MMIB4NFk9Io6iZoAfEBQfsbkhBMBwCu94XhAplkNok6P%2Bn5OzxIu8ygzlWeSlpofclIGozpjhT3q51gOqoSu%2FQESHdAPjis8avLprLsNKR2b4t3hAVWUgCnMWAAqDjgi4wBmkEsYYDUUxQlBVqXtVrGEeR0ehmeWtJmvw8M90EK%2Bzy5a1yVdN3LpKf%2FbKuiZi7z1XJQ7e%2B8QV6r%2FQozngc1aHKmOZS442GDWz9kMixQI1NdvxVd%2BqILAG09hjAR1K3qLYkADJk8dN57yqQM4h%2Fm99lk5kU2HNzIfPwSYNPr4uz2PxSNDXXdTLj1%2FTDVqs6cA9vmHozgkYko4EF3vNrSy4b0Lv%2FW3Da8xL1skZz4DNCa5N%2BiE2rSR2xconIoRd8MTBInfOlA6YinH5HgLvGMP2ndaDWrhHEcp0p%2F%2FKKziu53PdGLEEgsEmQnvRGbas2ry9Zfns6%2F1CvJ%2B13dvQsCeUMd4VoWd0jB03C5aYq3HUc1IKtSSLJQZncWk8e8Ttfjz6Fs%3D&nvMid=29030934589&ca

일렉트로룩스 ZB3311 133,170원 생활가전 청소기 무선청소기 품목 : 핸디스틱청소기|형태 : 상중심|집진방식 : 싸이클론|사용시간 : 1시간10분|청소방식 : 흡입전용|충전시간 : 2시간30분|배터리용량 : 2,500mAh|배터리종류 : 리튬이온|, , , , , , , https://cr.shopping.naver.com/adcr.nhn?x=pBR%2BkTnap24gqwebjASWyv%2F%2F%2Fw%3D%3DstA1vRmX69FYsurgYhFu641TBXaMG4SwJZboZ%2B85nNV%2B8DhkNlo8m633jJMNPmptHG726rDWBOK%2FHikMgjr7OXzyTea7ef%2Fk8q82Gur5QbWRgZZv%2B%2BYWgfyZEgzoTMCgZfXFoDUmy2FKJpYN76ZWyr0gAn8lD2%2BxrgblrBqYwPdWug%2B9sjA7rHdeMLkY%2BwIMU0DkUr4XYlJieRXSXepjJ94Rf8AF7SV93NQpFUK2GCu9UljeWTnmzwB%2BrN5HBCAeiWpmJuOgEl9qgM5iNvNejOL4rTAfmc328hgAILDBuhmmzdfsxYNfSpXYTyu05Hbpt334rL%2BbR36BhYe0R%2FrUmjgBSTS1sRv49oBHxm%2Bxq6HixayJxfyiqnjSP%2BitSH6kku75pppL%2F6qczMZ7Pf%2FUC8K4PAch8oPqbYMXypsXhHcJxIFuFfawWM4fFJOzfPpGOsv4Pv0kWpdQcKGzAgoBrl1ri7DS24rjIRy%2FX6qMfuShBXRl7pgdUN0UiuNAaHiPSVrUVrbpaPWWyyuX8K0%2FLjXn4CDjLWMgZO3PaiSnfLT4RpOFzCbaM6OU97uoPifjr%2B006p%2BtMF4i47raP0hqHm%2F0wu1mvh%2BHrLGqCQEskhTPNjqh%2FZemCCOboDhnweAmG&nvMid=13305164282&catId=50002350
툴앤툴 네오스틱 T1 109,000원 생활가전 청소기 무선청소기 품목 : 핸

LG전자 A9S 오브제컬렉션 AO9471WKT 1,059,000원 생활가전 청소기 무선청소기 품목 : 핸디스틱청소기|형태 : 상중심|집진방식 : 싸이클론|사용시간 : 1시간10분|청소방식 : 흡입전용|충전시간 : 2시간30분|배터리용량 : 2,500mAh|배터리종류 : 리튬이온|, , , , , , , https://cr.shopping.naver.com/adcr.nhn?x=KhS%2BPVvROhQblOpyTlc7Fv%2F%2F%2Fw%3D%3DsEbG42vPgdrxE%2FWwF8mzPFHHtcDeFrdHB87iJh9yYLmCJ5ltDmER3oclazeOpToBHolxzsAAHo1Uzzu5Wv60nsOnC97f9n9w2p8l05oUeJZxQm4SLs3i%2FjGGPR43wmWtwFGhnHFn6I2VJH6fQb%2FD%2BstyGaSr%2B0dRu37W7MMIB4NFk9Io6iZoAfEBQfsbkhBMB6ckgXskhJWvvT4MC6eusxoO0zSv5bqoBqynfRBikEohjhT3q51gOqoSu%2FQESHdAPjis8avLprLsNKR2b4t3hAVWUgCnMWAAqDjgi4wBmkEsYYDUUxQlBVqXtVrGEeR0ehmeWtJmvw8M90EK%2Bzy5a1yVdN3LpKf%2FbKuiZi7z1XJQkUJjIks6QADCsomBlm1873Po3fIaWlR4QSDZQrtJ4zld%2BqILAG09hjAR1K3qLYkCYz7SLnQVhlfbrd5%2F1Cc9BendIm%2Fo55usxyaJlq4i%2Be9FVD7lpvYYBCVan0AzPuRDGSnOm7w8AfkfDwBBGLFx3hpvOak3LKNNEkf3lc6KYSvI0fIFRfBZUPaSLRPGc3PLXNklCshdJqsRSDZtz8ENAyqc93PgHFEmLpEpxV%2BUSwo6%2F74bs%2FzNmznhIO5N5zjW6FXp2%2FAo1rMWKjYv38GbE4IU1JK9QVEvJFfErmQB71WhopkeKUdu5q4dSKA0zkoA%3D&nvMid=28125694523&catId=50002